<h1>CS4619: Artificial Intelligence II</h1>
<h1>Recommender Systems II</h1>
<h2>
    Derek Bridge<br />
    School of Computer Science and Information Technology<br />
    University College Cork
</h2>

<h1>Initialization</h1>
$\newcommand{\Set}[1]{\{#1\}}$ 
$\newcommand{\Tuple}[1]{\langle#1\rangle}$ 
$\newcommand{\v}[1]{\pmb{#1}}$ 
$\newcommand{\cv}[1]{\begin{bmatrix}#1\end{bmatrix}}$ 
$\newcommand{\rv}[1]{[#1]}$ 
$\DeclareMathOperator{\argmax}{arg\,max}$ 
$\DeclareMathOperator{\argmin}{arg\,min}$ 
$\DeclareMathOperator{\dist}{dist}$
$\DeclareMathOperator{\abs}{abs}$

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

<h1>MovieLens Dataset</h1>
<ul>
    <li>We'll illustrate this lecture using a widely available dataset.</li>
    <li>It has 
        <ul>
            <li>943 users, about whom it records some limited demographic data;</li>
            <li>1682 movies (items), about which it records some genres and some other data;</li>
            <li>100000 ratings on a 1-5 scale.</li>
        </ul>
    </li>
</ul>

In [3]:
# The code in this cell is used in Google's Recommender Systems course: https://developers.google.com/machine-learning/recommendation/

from urllib.request import urlretrieve
import zipfile

urlretrieve("http://files.grouplens.org/datasets/movielens/ml-100k.zip", "movielens.zip")
zip_ref = zipfile.ZipFile('movielens.zip', "r")
zip_ref.extractall()
print("Done. Dataset contains:")
print(zip_ref.read('ml-100k/u.info'))

# Load each data set (users, movies, and ratings).
users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    'ml-100k/u.user', sep='|', names=users_cols, encoding='latin-1')

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')

# The movies file contains a binary feature for each genre.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies_cols = [
    'movie_id', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols
movies = pd.read_csv(
    'ml-100k/u.item', sep='|', names=movies_cols, encoding='latin-1')

# Since the ids start at 1, we shift them to start at 0.
users["user_id"] = users["user_id"].apply(lambda x: str(x-1))
movies["movie_id"] = movies["movie_id"].apply(lambda x: str(x-1))
movies["year"] = movies['release_date'].apply(lambda x: str(x).split('-')[-1])
ratings["movie_id"] = ratings["movie_id"].apply(lambda x: str(x-1))
ratings["user_id"] = ratings["user_id"].apply(lambda x: str(x-1))
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))

# Compute the number of movies to which a genre is assigned.
genre_occurences = movies[genre_cols].sum().to_dict()

# Since some movies can belong to more than one genre, we create different
# 'genre' columns as follows:
# - all_genres: all the active genres of the movie.
# - genre: randomly sampled from the active genres.
def mark_genres(movies, genres):
  def get_random_genre(gs):
    active = [genre for genre, g in zip(genres, gs) if g==1]
    if len(active) == 0:
      return 'Other'
    return np.random.choice(active)
  def get_all_genres(gs):
    active = [genre for genre, g in zip(genres, gs) if g==1]
    if len(active) == 0:
      return 'Other'
    return '-'.join(active)
  movies['genre'] = [
      get_random_genre(gs) for gs in zip(*[movies[genre] for genre in genres])]
  movies['all_genres'] = [
      get_all_genres(gs) for gs in zip(*[movies[genre] for genre in genres])]

mark_genres(movies, genre_cols)

# Create one merged DataFrame containing all the movielens data.
movielens = ratings.merge(movies, on='movie_id').merge(users, on='user_id')

# Utility to split the data into training and test sets.
def split_dataframe(df, holdout_fraction=0.1):
  """Splits a DataFrame into training and test sets.
  Args:
    df: a dataframe.
    holdout_fraction: fraction of dataframe rows to use in the test set.
  Returns:
    train: dataframe for training
    test: dataframe for testing
  """
  test = df.sample(frac=holdout_fraction, replace=False)
  train = df[~df.index.isin(test.index)]
  return train, test

Done. Dataset contains:
b'943 users\n1682 items\n100000 ratings\n'


In [4]:
# Some hackery to insert some extra features

# One-hot encode the sex
movielens['female'] = (movielens['sex'] == 'F').astype(int)
movielens['male'] = (movielens['sex'] == 'M').astype(int)

# Genres
item_features = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

# User demographic features that we will use
user_features = ['age', 'female', 'male']

# Insert interaction features
interaction_features = [ifeat + '_' + ufeat for ifeat in item_features for ufeat in user_features]
for ifeat in item_features:
    for ufeat in user_features:
        movielens[ifeat + '_' + ufeat] = movielens[ifeat] * movielens[ufeat]

In [5]:
# Training and test sets
train, test = split_dataframe(movielens, holdout_fraction=0.2)

movielens_by_user_train = train[['user_id', 'movie_id', 'rating']].pivot_table(
    index=['user_id'], columns=['movie_id'], values='rating', fill_value=0)

<h1>User-Item Interactions</h1>
<ul>
    <li>Recall the ratings matrix, $\v{R}$:
        <table style="border: 1px solid; border-collapse: collapse;">
            <tr>
                <th style="border: 1px solid black; text-align: left;"></th>
                <th style="border: 1px solid black; text-align: left;">$i_1$</th>
                <th style="border: 1px solid black; text-align: left;">$i_2$</th>
                <th style="border: 1px solid black; text-align: left;">$i_3$</th>
                <th style="border: 1px solid black; text-align: left;">$i_4$</th>
                <th style="border: 1px solid black; text-align: left;">$i_5$</th>
                <th style="border: 1px solid black; text-align: left;">$i_6$</th>
            </tr>
            <tr>
                <th style="border: 1px solid black; text-align: left;">$u_1$</th>
                <td style="border: 1px solid black; text-align: left;"></td>
                <td style="border: 1px solid black; text-align: left;">2</td>
                <td style="border: 1px solid black; text-align: left;">5</td>
                <td style="border: 1px solid black; text-align: left;">3</td>
                <td style="border: 1px solid black; text-align: left;">1</td>
                <td style="border: 1px solid black; text-align: left;">2</td>
            </tr>
            <tr>
                <th style="border: 1px solid black; text-align: left;">$u_2$</th>
                <td style="border: 1px solid black; text-align: left;">5</td>
                <td style="border: 1px solid black; text-align: left;">5</td>
                <td style="border: 1px solid black; text-align: left;"></td>
                <td style="border: 1px solid black; text-align: left;">3</td>
                <td style="border: 1px solid black; text-align: left;">4</td>
                <td style="border: 1px solid black; text-align: left;"></td>
            </tr>
            <tr>
                <th style="border: 1px solid black; text-align: left;">$u_3$</th>
                <td style="border: 1px solid black; text-align: left;"></td>
                <td style="border: 1px solid black; text-align: left;"></td>
                <td style="border: 1px solid black; text-align: left;"></td>
                <td style="border: 1px solid black; text-align: left;"></td>
                <td style="border: 1px solid black; text-align: left;">3</td>
                <td style="border: 1px solid black; text-align: left;"></td>
            </tr>
            <tr>
                <th style="border: 1px solid black; text-align: left;">$u_4$</th>
                <td style="border: 1px solid black; text-align: left;">5</td>
                <td style="border: 1px solid black; text-align: left;">4</td>
                <td style="border: 1px solid black; text-align: left;">2</td>
                <td style="border: 1px solid black; text-align: left;">4</td>
                <td style="border: 1px solid black; text-align: left;">3</td>
                <td style="border: 1px solid black; text-align: left;">3</td>
            </tr>
            <tr>
                <th style="border: 1px solid black; text-align: left;">$u_5$</th>
                <td style="border: 1px solid black; text-align: left;">2</td>
                <td style="border: 1px solid black; text-align: left;">5</td>
                <td style="border: 1px solid black; text-align: left;">4</td>
                <td style="border: 1px solid black; text-align: left;">4</td>
                <td style="border: 1px solid black; text-align: left;"></td>
                <td style="border: 1px solid black; text-align: left;"></td>
            </tr>
        </table>
    </li>
   <li>We made no use of this in the simple content-based recommender system that we presented in the previous
       lecture.
    </li>
    <li>In this lecture and the next, we will see how we can make use of this data.</li>
</ul>

<h2>Rating scales</h2>
<ul>
    <li>Before we use $\v{R}$, let's note a couple of problems.
        <ul>
            <li>When users give ratings, some users use the whole scale (1-5); others do not (e.g. reluctant to give a 5; or reluctant to give a 1).</li>
            <li>Some users are more positive (mostly they give 4's and 5's); others are more negative (their ratings are skewed away from the top).</li>
        </ul>
        This raises issues about whether different rows of $\v{R}$ are comparable.
    </li>
    <li>Solutions:
        <ul>
            <li>Some recommender systems already takes steps to compensate for these differences in users' ratings scales (e.g. Pearson correlation in the user-based nearest-neighbours recommender below).</li>
            <li>Otherwise, we can standardize each user's ratings.</li>
        </ul>
    </li>
    <li>There are other problems with ratings too. Can you think of any?
        <!-- Opinions change over time because tastes change over time. Opinions may need recalibration when
             new items come along. Ratings are given some time after consumption: the delay reduces reliability.
             Re-rating tasks show that users have some inconsistency.
             Item ratings tend towards the mean.
        -->
    </li>
</ul>        

<h1>Non-Personalised Recommender Systems</h1>
<ul>
    <li>Using $\v{R}$, we can define some non-personalised recommender systems.</li>
    <li>The obvious one is: $$\hat{r}_{ui} = \frac{\sum_{r \in \v{R}_i} r}{|\v{R}_i|}$$
        where $\v{R}_i = \{r_{vi} : r_{vi} \neq \bot, \forall v \in U\}$ What does this recomender do?</li>
    <li>This turns out to be a great baseline to compare against &mdash; quite hard to beat.</i>
</ul>

In [8]:
# Fit: precompute the mean rating and the mean rating for every item in the training data
mean_rating = train['rating'].mean()
mean_rating_by_item = train.groupby('movie_id').mean()['rating']

# Predict: return mean rating for the item or mean of all ratings
def predict(df):
    return df['movie_id'].apply(
        lambda i: mean_rating_by_item.iloc[int(i)] if i in mean_rating_by_item.index else mean_rating)

# Testing (error estimation)
mean_absolute_error(test['rating'], predict(test))

1.216151909776916

<h1>Supervised Content-Based Recommender System</h1>
<ul>
    <li>If we are going to do supervised learning, then we need a labeled dataset: we need target values.</li>
    <li>Let's use the ratings as the targets in a regression task.</li>
    <li>What about the features?</li>
    <li>Items:
        <ul>
            <li>We can describe each item $i$ the same way as in the simple content-based recommender: a vector
                $\v{Q}^{(i)}$ whose dimension is $d$, where the features are genres, for example.
            </li>
        </ul>
    </li>
    <li>Users:
        <ul>
            <li>In the simple content-based recommender, we described each user $u$ using a vector $\v{P}^{(u)}$
                in the same space, hence having the same dimension $d$ and the same features, e.g. genres.
            </li>
            <li>But this is no longer necessary because we're not doing item-user similarities anymore.</li>
            <li>So $\v{P}^{(u)}$ can be of dimension $d'$, where $d'$ is not necessarily the same as $d$.</li>
            <li>And the features can be genres, but they can be other things instead, or as well. For example,
                we can have demographic features such as age and sex.
            </li>
        </ul>
    </li>
    <li>Interaction features:
        <ul>
            <li>And we can have interaction features (remember these from feature engineering?).</li>
            <li>This gives us $d \times d'$ additional features.</li>
        </ul>
    </li>
    <li>(Note that the ratings are not used as features. Contrast this with collaborative recommender 
        systems below.)
    </li>
</ul>

<h2>Linear regression</h2>
<ul>
    <li>Using the features from above, we can learn any model we like.</li>
    <li>Most obvious is a linear model, hence we have linear regression.</li>
    <li>If we assume that there are $d=3$ item features and $d' = 2$ user features, then the predicted ratingis
        $$\hat{r}_{ui} = \v{\beta}_0 +
                        \v{\beta}_1\v{P}^{(u)}_1 +
                        \v{\beta}_2\v{P}^{(u)}_2 +
                        \v{\beta}_3\v{Q}^{(i)}_1 +
                        \v{\beta}_4\v{Q}^{(i)}_2 +
                        \v{\beta}_5\v{Q}^{(i)}_3 +
                        \v{\beta}_6\v{P}^{(u)}_1\v{Q}^{(i)}_1 +
                        \v{\beta}_7\v{P}^{(u)}_1\v{Q}^{(i)}_2 +
                        \v{\beta}_8\v{P}^{(u)}_1\v{Q}^{(i)}_3 +
                        \v{\beta}_9\v{P}^{(u)}_2\v{Q}^{(i)}_1 +
                        \v{\beta}_{10}\v{P}^{(u)}_2\v{Q}^{(i)}_2 +
                        \v{\beta}_{11}\v{P}^{(u)}_2\v{Q}^{(i)}_3$$
    </li>
    <li>Now we can use OLS regression!</li>
    <li>The number of parameters here is $1 + d' + d + d'd$ and this might cause various problems but we already know some solutions (e.g.
        dimensionality reduction, regularization, &hellip;).
    </li>
    <li>Once we have learned the linear model then, for a user $u$, we can predict a rating for each candidate and
        use these as the scores to order the candidates.
    </li>
</ul>

In [10]:
# Features for the linear regression
features = item_features + user_features + interaction_features

# Training
lr = LinearRegression()
lr.fit(train[features], train['rating'])

# Testing (error estimation)i
mean_absolute_error(test['rating'], lr.predict(test[features]))

0.915039417757867

<ul>
    <li>This is only a little bit better than the non-personalised recommender.</li>
    <li>We could probably improve it with more features of perhaps regularization, etc.
        We should also probably be rounding up/down any out-of-range predictions so that they fall within $[1,5]$,
        and possibly even rounding each individual prediction to the nearest integer.
    </li>
    <li>We should have done some model selection with a validation set but, to keep the code simple, we
        ignored this.
    </li>
</ul>

<h1>Collaborative Filtering</h1>
<ul>
    <li>Above, we used a user's ratings as her target values.</li>
    <li>Here, we will use ratings as both target values and features.</li>
    <li>We'll look at two more recommender systems. 
        <ul>
            <li>User-based nearest-neighbours;</li>
            <li>Matrix factorization.</li>
        </ul>
        The first is instance-based; the second (next lecture) is model-based.
    </li>
    <li>Both are examples of <b>collaborative filtering</b>:
        <ul>
            <li>To predict $\hat{r}_{ui}$, use other people's opinions of $i$.
        </ul>
    </li>
</ul>

<h1>User-Based Nearest-Neighbours</h1>
<ul>
    <li>The idea is simple. To predict $\hat{r}_{ui}$,
        <ul>
            <li>find $k$ people who are similar to $u$ (the nearest-neighbours);</li>
            <li>take the mean of the neighbours' ratings for $i$.</li>
        </ul>
    </li>
</ul>

<h2>User-based nearest-neighbours: some details</h2>
<ul>
    <li>How do we compute similarity of two users, $u$ and $v$?
        <ul>
            <li>We take their vectors of ratings.
                <ul>
                    <li>e.g. $u_2$ is represented by $\rv{5,5,\bot,3,4,\bot}$</li>
                    <li>e.g. $u_3$ is represented by $\rv{2,5,4,4,\bot,\bot}$</li>
                </ul>
            </li>
            <li>We compute cosine similarity, for example.
                <ul>
                    <li>In fact, Pearson correlation (details not important), which lies in 
                        $[-1,+1]$ is more common. It has the advantage that it takes into account some of the
                        variation in users use of the rating scale that we mentioned above.
                    </li>
                    <li>In fact, modifications of Pearson correlation (details not important) are used to adjust 
                        for the case where $u$ and $v$ have
                        few co-rated items and to deal with some edge cases.
                    </li>
                </ul>
            </li>
        </ul>
    </li>
    <li>How many neighbours, $k$?
        <ul>
            <li>This is a hyperparameter, whose values can be chosen through a grid-search. Values of 50 or 
                100 are common.
            </li>
            <li>However, there are variants (details not important). For example, instead of finding the $k$ 
                most similar neighbours, we might find all neighbours whose similarity to $u$ exceeds some 
                threshold $\theta$. Or we might include a constraint too: e.g. we only include neighbours 
                who have a rating for candidate $i$.
            </li>
        </ul>
    </li>
    <li>How do we make the prediction?
        <ul>
            <li>Simplest is just the mean of the neghbours' ratings for $i$.</li>
            <li>But we may want a weighted mean, where weights are similarities. And, if using Pearson, we may
                want to exclude neighbours whose similarity to $u$ is negative. And we may need to handle 
                edge cases, such as where none of the neighbours has rated $i$.
            </li>
        </ul>
    </li>     
</ul>

In [32]:
# Don't rely on this implementation!
# On the one hand, it is over-complicated because of the wrangling it has to do to the dataframes.
# On the other hand, it makes many simplifications, e.g.: 
#    it uses dot product (unnormalised cosine) as the similarity measure instead, e.g. of a variant of Pearson
#    it predicts the mean of the neighbours ratings, rather than a similarity-weighted mean 
#    if no neighbour has a rating for the item, it predicts the mean of all ratings rather than, e.g. the mean 
#                                                                                       of that user's ratings
# and so on.

def sim(u_ratings, v_ratings):
    return u_ratings.dot(v_ratings)

def user_based_nn_collaborative_filter(u, i, ratings_by_user, k=50):
    # No ratings for i at all
    if i not in ratings_by_user.columns:
        return mean_rating
    # u's ratings
    u_ratings = ratings_by_user.loc[u]
    # Find which k users in R are the most similar to u (excluding u herself)
    nbrs = sorted([(v, sim(u_ratings, ratings_by_user.loc[v])) 
                   for v in ratings_by_user.index if u != v], key=lambda x: x[1])[::-1][:k]
    # Return the corresponding mean target value from R
    nbrs_ratings = ratings_by_user.loc[[v for v, sim in nbrs]][i]
    num_nbrs_ratings = np.count_nonzero(nbrs_ratings)
    return nbrs_ratings.sum() / num_nbrs_ratings if num_nbrs_ratings > 0 else mean_rating 

In [34]:
# Warning - takes some time to run!

predictions = [user_based_nn_collaborative_filter(u, i, movielens_by_user_train) for (u, i) in test[['user_id', 'movie_id']].values]
mean_absolute_error(test['rating'], predictions)

0.8194158637959997

<h2>Discussion of user-based nearest-neighbours</h2>
<ul>
    <li>Advantages of user-based nearest-neighbours collaborative filtering include:
        <ul>
            <li>It does not require any item or user descriptions, just user-item interactions (e.g. ratings) &mdash;
                and this is data we will collect during the normal operation of the system.
            </li>
            <li>It may recommend items that are pleasantly surprising (certainly more so than content-based
                approaches), since it recommends using <em>other peoples'</em> tastes.
            </li>
            <li>New ratings can take immediate effect.</li>
        </ul>
    </li>
    <li>Its disadvantages include:
        <ul>
            <li>It is slow at prediction time. (It can be sped up by caching some of the computations
                during 'fitting'. But this may work in opposition to the point above about new ratings taking
                immediate effect.)
            </li>
            <li>It has problems recommending to cold-start users and recommending cold-start items.</li>
            <li>It can exhibit popularity bias: over-recommending popular items (although this may depend to
                some extent on details of the implementation).
            </li>
        </ul>
    </li>
    <li>Can it explain its recommendations?
        <ul>
            <li>Some people say, No. Displaying the identities of the active user’s neighbours is unlikely to be 
                effective, since the user will in general not know the neighbours; displaying their ratings is 
                unlikely to be effective, since even the ratings they have in common with the user will be too 
                large to be readily comprehended.
            </li>
            <li>Some people say, Yes. You can summarize the neighbours' opinons, e.g. using a histogram that shows
                how many of the $k$ neighbours awarded the item a 5, how many a 4, and so on. 
                (I have also published work that gives another way of explaining user-based recommendations.)
            </li>
        </ul>
    </li>
    <li>In concluding, let's mention that it is also possible to build <em>item-based</em> nearest-neighbours collaborative
        filters.
        <ul>
            <li>Without going into details, these use the similarities between item ratings (columns in the
                matrix, rather than rows).
            </li>
            <li>Famously, this is one of the methods used by Amazon.</li>
        </ul>
    </li>
</ul>